TP n°2 : prédiction à l'échelle du pixel - cas faiblement supervisés

Notions abordées:

En partie I:
- Test d'un FCN-ResNet sur un problème de segmentation sémantique. Métriques adaptées.
- Appentissage parfaitement supervisé d'un U-Net sur un problème de débruitage. 

En partie II:
- Le scénario noise-to-noise
- Neural Eggs Separation (cas simplifié)

Durée : 2 h

## Partie II: débruitage d'images avec un FCN - cas faiblement supervisés




In [ ]:
#imports de base et montage du drive
import matplotlib.pyplot as plt
import numpy as np
import os
import time

import torch
import torchvision
import torch.nn as nn   
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import torch.optim as optim
from PIL import Image

Monter le drive pour pouvoir importer **utile.py**. Changer le répertoire de travail à cet effet.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#On se branche au répertoire contenant utile.py:
os.listdir()
os.chdir('drive/MyDrive/TP_ENM_2223/TP_2223_corr_P')

In [ ]:
os.listdir() 
from utile import *

###Exercice 1: le scénario "noise to noise"

Dans l'exercice 2 partie I, nous disposions de paires (version bruitée , version propre) sur lequelles apprendre.
Mais dans la réalité, on ne dispose pas toujours de versions propres.  \\
Par contre, il arrive que plusieurs versions bruitées soient disponibles. Penser par exemple à des photos prises avec un téléobjectif à plusieurs secondes d'écart: les effets de la turbulence de l'air sur la qualité de l'image sont indépendants d'une image à l'autre.  \\
Dans le scénario "noise to noise" [(Lehtinen,2018)](https://arxiv.org/pdf/1803.04189.pdf), on dispose de paires d'images bruitées indépendamment, rangées dans deux jeux $B^1_{noisy}$ et $B^2_{noisy}$.  
Pour nous placer dans ce scénario, nous reprenons les images de synthèse de l'exercice 1. La fonction *gen_noise2noise* permet d'échantillonner les deux bases:

In [ ]:
image1, image2 = gen_noise2noise(6)
fig = plt.figure(0, figsize=(36, 6))
voir_batch2D(image1, 6, fig, k=0, min_scale=0,max_scale=1)

fig2 = plt.figure(1, figsize=(36, 6))
voir_batch2D(image2.detach().cpu(), 6, fig2, k=0, min_scale=0,max_scale=1)

**Q1** Définir une procédure d'apprentissage qui permette de débruiter convenablement l'image. On se laissera guider par l'intuition dans un premier temps.

In [ ]:
fcn = UNet(1,1,16).cuda()  #1 canal entrée, 1 canal de sortie, paramètre taille du réseau: 16

import torch.optim as optim
optimizer = optim.Adam(fcn.parameters(), 10**(-3))

In [ ]:
def criterion(output,target):
  return ...

nepochs = 40 #réfléchir à la suite en attendant...
nbatches = 100
batchsize = 64

train_losses = []


for epoch in range(nepochs):
    

    print("Epoch " + str(epoch))
    epoch_losses  = []
    for i in range(nbatches):  
        #Load inputs
        
        # Génération des images

        # Passage sur GPU

        # mise à zéro des gradients, forward pass

        # coût et gradients
        
        # MAJ des poids

        epoch_losses.append(loss.detach().cpu())

        del target, input, loss
        torch.cuda.empty_cache()     

    epoch_loss = np.mean(epoch_losses)
    train_losses.append(epoch_loss)    
    print('epoch loss : \n')
    print(epoch_loss)

In [ ]:
plt.plot(train_losses)

In [ ]:
#visualisation:

fcn.eval()

input, target = gen_noise2noise(6)

output = fcn(input.cuda())


fig = plt.figure(0, figsize=(36, 6))
voir_batch2D(input, 6, fig, k=0, min_scale=0,max_scale=1)  #entrées
fig2 = plt.figure(1, figsize=(36, 6))
voir_batch2D(target, 6, fig2, k=0, min_scale=0,max_scale=1)  #cibles
fig3 = plt.figure(2, figsize=(36, 6))
voir_batch2D(output.detach().cpu(), 6, fig2, k=0, min_scale=0,max_scale=1) #sorties


**Q2** analyse théorique:

La procédure d'entraînement revient à chercher les poids $\theta^*$ vérifiant:  
\begin{equation}
\theta^* = \underset{\theta}{argmin} {\big \{} \mathbb{E}_{(X,Y)}[ \ \mathcal{L} (f_\theta(X), Y) \ ] {\big \}} \tag{1}
\end{equation}

Réécrire l'équation (1) pour justifier la démarche suivie en **Q1**.



On réécrit l'espérance à minimiser dans le cas d'une fonction de coût égale à la MAE:
\begin{equation}
 \mathbb{E}_{(X,Y)}[ \ \mathcal{L} (f_\theta(X), Y) \ ] = \mathbb{E}_{X}[ \mathbb{E}_{Y | X}\ |f_\theta(X) - Y | \ ]
\end{equation}

Pour atteindre le minimum, le réseau doit fournir la médiane conditionnelle de $Y$ sachant $X$. Dans le cas d'une fonction de coût quadratique, le réseau proposera l'espérance conditionnelle. Contrairement à la médiane, celle-ci est biaisée, du fait que les éléments parasites sont toujours positifs.


###Exercice 2: le scénario "NES" (Neural Egg Separation)
Dans un autre scénario, deux sources d'images peuvent être échantillonnées indépendamment. La première fournit des images bruitées contenant l'objet d'intérêt. Les images provenant de la seconde source ne contiennent que du bruit. De plus, le processus de corruption, c'est à dire la façon dont le bruit est combiné à l'image propre, est supposé connu.
C'est un scénario relativement fréquent. Dans le cas d'images radar météorologique, par exemple, il est possible d'extraire des images ne contenant que du bruit en dehors des périodes de précipitations.  
Ce scénario a été défini par [Halperin et al (2018)](https://arxiv.org/pdf/1811.12739.pdf). Ces auteurs proposent une méthode itérative pour le traiter (Neural Egg Separation). Dans cet exercice, on s'appuiera sur un principe assez proche, toujours avec nos images de synthèse. 

Les fonctions *gen1_NES* et *gen2_NES* permettent d'échantillonner les deux sources. Le processus de corruption est trivial: le bruit (rectangles) est simplement sommé à l'image propre (cellules).   



In [ ]:
image1 = gen1_NES(6)
noise = gen2_NES(6)

#échantilloner la source d'images bruitées:
fig = plt.figure(0, figsize=(36, 6))
voir_batch2D(image1, 6, fig, k=0, min_scale=0,max_scale=1)

#échantilloner la source d'images ne contenant que du bruit:
fig2 = plt.figure(1, figsize=(36, 6))
voir_batch2D(noise, 6, fig2, k=0, min_scale=0,max_scale=1)

**Q1** Proposer une stratégie d'apprentissage qui débouche sur un débruitage efficace.

**Q2** Discuter les limites de la méthode et évoquer quelques voies d'amélioration.